# C2C Marketplace Retention Analysis

This notebook performs advanced statistical analysis on user retention and A/B test results.

## Objectives
1. Funnel analysis and drop-off identification
2. Cohort retention analysis
3. A/B test statistical significance testing
4. User segment behavior comparison

## Setup and Authentication

In [ ]:
# Install required packages (run once)
!pip install google-cloud-bigquery pandas numpy matplotlib seaborn scipy statsmodels

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.colab import auth
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
# Authenticate with Google Cloud
auth.authenticate_user()

# Set your project ID
PROJECT_ID = 'your-project-id'  # CHANGE THIS
DATASET_ID = 'analytics'

# Create BigQuery client
client = bigquery.Client(project=PROJECT_ID)
print(f"Connected to project: {PROJECT_ID}")

## 1. Data Loading

In [ ]:
# Load users data
users_query = f"""
SELECT *
FROM `{PROJECT_ID}.{DATASET_ID}.users`
"""

users_df = client.query(users_query).to_dataframe()
print(f"Loaded {len(users_df)} users")
users_df.head()

In [ ]:
# Load events data
events_query = f"""
SELECT *
FROM `{PROJECT_ID}.{DATASET_ID}.events`
"""

events_df = client.query(events_query).to_dataframe()
print(f"Loaded {len(events_df)} events")
events_df.head()

## 2. Funnel Analysis

In [ ]:
# Calculate funnel metrics
funnel_counts = events_df['event_type'].value_counts().sort_index()

funnel_order = ['page_view', 'search', 'item_view', 'chat_click', 'chat_send']
funnel_data = [funnel_counts.get(event, 0) for event in funnel_order]

# Calculate conversion rates
conversion_rates = []
for i in range(len(funnel_data) - 1):
    if funnel_data[i] > 0:
        rate = (funnel_data[i + 1] / funnel_data[i]) * 100
        conversion_rates.append(rate)
    else:
        conversion_rates.append(0)

# Display funnel
funnel_df = pd.DataFrame({
    'Stage': funnel_order,
    'Count': funnel_data,
    'Conversion Rate': ['-'] + [f"{rate:.1f}%" for rate in conversion_rates]
})

print("\n=== Funnel Analysis ===")
print(funnel_df.to_string(index=False))

In [ ]:
# Visualize funnel
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Funnel chart
ax1.barh(funnel_order, funnel_data, color='steelblue')
ax1.set_xlabel('Number of Events')
ax1.set_title('User Journey Funnel')
ax1.invert_yaxis()

for i, (stage, count) in enumerate(zip(funnel_order, funnel_data)):
    ax1.text(count, i, f' {count:,}', va='center')

# Conversion rates
stages_with_rates = [f"{funnel_order[i]} → {funnel_order[i+1]}" for i in range(len(conversion_rates))]
colors = ['red' if rate < 30 else 'orange' if rate < 60 else 'green' for rate in conversion_rates]

ax2.barh(stages_with_rates, conversion_rates, color=colors)
ax2.set_xlabel('Conversion Rate (%)')
ax2.set_title('Stage-to-Stage Conversion Rates')
ax2.invert_yaxis()

for i, rate in enumerate(conversion_rates):
    ax2.text(rate, i, f' {rate:.1f}%', va='center')

plt.tight_layout()
plt.show()

## 3. A/B Test Analysis

In [ ]:
# Filter for A/B test participants only
ab_events = events_df[events_df['ab_group'].isin(['control', 'treatment'])].copy()

# Calculate chat click rate by group
ab_analysis = ab_events.groupby('ab_group').agg({
    'event_id': 'count',
    'user_id': 'nunique'
}).rename(columns={'event_id': 'total_events', 'user_id': 'unique_users'})

# Count chat clicks by group
chat_clicks = ab_events[ab_events['event_type'] == 'chat_click'].groupby('ab_group').size()
item_views = ab_events[ab_events['event_type'] == 'item_view'].groupby('ab_group').size()

ab_analysis['item_views'] = item_views
ab_analysis['chat_clicks'] = chat_clicks
ab_analysis['chat_click_rate'] = (ab_analysis['chat_clicks'] / ab_analysis['item_views'] * 100).round(2)

print("\n=== A/B Test Results ===")
print(ab_analysis)

In [ ]:
# Statistical significance test (Chi-square test)
control_clicks = ab_analysis.loc['control', 'chat_clicks']
control_views = ab_analysis.loc['control', 'item_views']
treatment_clicks = ab_analysis.loc['treatment', 'chat_clicks']
treatment_views = ab_analysis.loc['treatment', 'item_views']

# Create contingency table
contingency_table = np.array([
    [control_clicks, control_views - control_clicks],
    [treatment_clicks, treatment_views - treatment_clicks]
])

chi2, p_value, dof, expected = stats.chi2_contingency(contingency_table)

# Calculate lift
control_rate = ab_analysis.loc['control', 'chat_click_rate']
treatment_rate = ab_analysis.loc['treatment', 'chat_click_rate']
lift = ((treatment_rate - control_rate) / control_rate * 100)

print("\n=== Statistical Test Results ===")
print(f"Control Rate: {control_rate:.2f}%")
print(f"Treatment Rate: {treatment_rate:.2f}%")
print(f"Lift: {lift:+.2f}%")
print(f"Chi-square statistic: {chi2:.4f}")
print(f"P-value: {p_value:.4f}")
print(f"\nResult: {'SIGNIFICANT' if p_value < 0.05 else 'NOT SIGNIFICANT'} (α = 0.05)")

In [ ]:
# Visualize A/B test results
fig, ax = plt.subplots(figsize=(10, 6))

groups = ['Control', 'Treatment']
rates = [control_rate, treatment_rate]
colors = ['steelblue', 'coral']

bars = ax.bar(groups, rates, color=colors, alpha=0.7, edgecolor='black')
ax.set_ylabel('Chat Click Rate (%)')
ax.set_title('A/B Test: Chat Click Rate Comparison')
ax.set_ylim(0, max(rates) * 1.2)

# Add value labels
for bar, rate in zip(bars, rates):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{rate:.2f}%',
            ha='center', va='bottom', fontsize=12, fontweight='bold')

# Add significance annotation
if p_value < 0.05:
    ax.text(0.5, max(rates) * 1.1, f'p = {p_value:.4f} *',
            ha='center', fontsize=10, color='red')
    ax.text(0.5, max(rates) * 1.15, f'Lift: {lift:+.1f}%',
            ha='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

## 4. Cohort Retention Analysis

In [ ]:
# Prepare data for cohort analysis
user_events = events_df.merge(users_df[['user_id', 'join_date', 'verified_neighborhood']], on='user_id')
user_events['event_date'] = pd.to_datetime(user_events['event_timestamp']).dt.date
user_events['join_date'] = pd.to_datetime(user_events['join_date'])
user_events['days_since_join'] = (pd.to_datetime(user_events['event_date']) - user_events['join_date']).dt.days

# Calculate retention by verification status
retention_data = []

for verified in [True, False]:
    cohort = user_events[user_events['verified_neighborhood'] == verified]
    
    for day in [1, 7, 14, 30]:
        total_users = cohort['user_id'].nunique()
        retained_users = cohort[cohort['days_since_join'] >= day]['user_id'].nunique()
        retention_rate = (retained_users / total_users * 100) if total_users > 0 else 0
        
        retention_data.append({
            'Verified': 'Yes' if verified else 'No',
            'Day': f'D+{day}',
            'Retention Rate': retention_rate
        })

retention_df = pd.DataFrame(retention_data)
retention_pivot = retention_df.pivot(index='Verified', columns='Day', values='Retention Rate')

print("\n=== Retention by Neighborhood Verification ===")
print(retention_pivot.round(2))

In [ ]:
# Visualize retention heatmap
plt.figure(figsize=(10, 4))
sns.heatmap(retention_pivot, annot=True, fmt='.1f', cmap='RdYlGn', 
            cbar_kws={'label': 'Retention Rate (%)'}, vmin=0, vmax=100)
plt.title('User Retention Heatmap by Neighborhood Verification')
plt.xlabel('Days Since Join')
plt.ylabel('Neighborhood Verified')
plt.tight_layout()
plt.show()

## 5. User Segment Analysis

In [ ]:
# Analyze behavior by user segment
segment_events = events_df.merge(users_df[['user_id', 'user_segment']], on='user_id')

segment_analysis = segment_events.groupby('user_segment').agg({
    'event_id': 'count',
    'user_id': 'nunique'
}).rename(columns={'event_id': 'total_events', 'user_id': 'users'})

segment_analysis['events_per_user'] = (segment_analysis['total_events'] / segment_analysis['users']).round(2)

# Calculate chat click rate by segment
for segment in segment_analysis.index:
    segment_data = segment_events[segment_events['user_segment'] == segment]
    item_views = len(segment_data[segment_data['event_type'] == 'item_view'])
    chat_clicks = len(segment_data[segment_data['event_type'] == 'chat_click'])
    
    segment_analysis.loc[segment, 'chat_click_rate'] = (chat_clicks / item_views * 100) if item_views > 0 else 0

print("\n=== User Segment Analysis ===")
print(segment_analysis.round(2))

In [ ]:
# Visualize segment comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Events per user
segment_analysis['events_per_user'].plot(kind='bar', ax=ax1, color='steelblue', alpha=0.7)
ax1.set_title('Average Events per User by Segment')
ax1.set_xlabel('User Segment')
ax1.set_ylabel('Events per User')
ax1.tick_params(axis='x', rotation=45)

# Chat click rate
segment_analysis['chat_click_rate'].plot(kind='bar', ax=ax2, color='coral', alpha=0.7)
ax2.set_title('Chat Click Rate by Segment')
ax2.set_xlabel('User Segment')
ax2.set_ylabel('Chat Click Rate (%)')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 6. Key Insights and Recommendations

In [ ]:
print("\n" + "="*60)
print("KEY INSIGHTS")
print("="*60)

print("\n1. FUNNEL BOTTLENECK")
print(f"   - Item View → Chat Click conversion: {conversion_rates[2]:.1f}%")
print("   - This is the primary drop-off point")

print("\n2. A/B TEST RESULTS")
print(f"   - Treatment group lift: {lift:+.1f}%")
print(f"   - Statistical significance: {'YES' if p_value < 0.05 else 'NO'} (p={p_value:.4f})")

print("\n3. NEIGHBORHOOD VERIFICATION IMPACT")
verified_d7 = retention_pivot.loc['Yes', 'D+7']
unverified_d7 = retention_pivot.loc['No', 'D+7']
print(f"   - D+7 Retention (Verified): {verified_d7:.1f}%")
print(f"   - D+7 Retention (Unverified): {unverified_d7:.1f}%")
print(f"   - Difference: {verified_d7 - unverified_d7:+.1f}pp")

print("\n4. USER SEGMENT INSIGHTS")
best_segment = segment_analysis['chat_click_rate'].idxmax()
print(f"   - Highest engagement segment: {best_segment}")
print(f"   - Chat click rate: {segment_analysis.loc[best_segment, 'chat_click_rate']:.1f}%")

print("\n" + "="*60)
print("RECOMMENDATIONS")
print("="*60)
print("\n1. Implement trust indicators on item detail pages")
print("2. Encourage neighborhood verification for new users")
print("3. Target high-engagement segments with personalized features")
print("4. A/B test additional UI improvements to boost chat clicks")
print("\n" + "="*60)